In [41]:
######### Check the notes file for simplified details
######### Check the other notebook for the simplified example with some extra options

##### Note: This file is more for backend stuff for KFold (doesnt use the cross_val_score function until the end, and without cv parameter)



from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits

import warnings
warnings.filterwarnings(action='ignore')

digits = load_digits()

In [6]:
### split data: (remember, this isnt needed for the actual cross_val_score function)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3)

# The problem here (using this function above) is that the result sets are not static
# Everytime you run this the sets will change, effecting the accuracy of each model


In [51]:
# First classifer is going to be LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.9115191986644408

In [52]:
# Support Vector Machines
svm = SVC()
svm.fit(X_train,y_train)
svm.score(X_test,y_test)

0.4273789649415693

In [53]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.9081803005008348

In [10]:
## Each time you change the datasets, results change, that's the problem, need to run multiple times to get an average

#### Now try KFold
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf


KFold(n_splits=3, random_state=None, shuffle=False)

In [12]:
# Model is ready, check to see how the iteration works, with some simple data you load into it
# Look at how it divides the data into 3 folds (iteratively), and each fold is split into its own train/test
# Each iteration is a 'split' (3) and each split is divided into 3 folds (2 for training, 1 for testing)
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


In [40]:
# Write a method/function to simplify the train/test for each model:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)
    
## test the function on LogisticRegression
get_score(LogisticRegression(),X_train, X_test, y_train, y_test)




0.9115191986644408

In [14]:
## Now use KFold with the function created above
# Use "Stratified KFold" here which divides each fold in a unified way
# This is like the stratified train/test splitting which works well for smaller datasets
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)  #usually people use 10 splits


In [39]:
# prepare the scores arrays
scores_l = []
scores_svm = []
scores_rf = []

# Do cross validation manually (this is NOT how to do it in real life, this is just to show what is happening)
# same as for kf example above (in cell #12)
for train_index,test_index in kf.split(digits.data):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                        digits.target[train_index], digits.target[test_index]
    scores_l.append(get_score(LogisticRegression(),X_train, X_test, y_train, y_test))
    scores_svm.append(get_score(SVC(),X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40),X_train, X_test, y_train, y_test))

In [25]:

# Check scores for each model: 
scores_l

[0.8964941569282137, 0.9515859766277128, 0.9115191986644408]

In [26]:
scores_svm

[0.41068447412353926, 0.41569282136894825, 0.4273789649415693]

In [27]:
scores_rf

[0.9465776293823038, 0.9432387312186978, 0.9265442404006677]

In [ ]:
# Get average of each model and can optimize models with parameters:


In [36]:

# Now do the same thing using in-built Cross Validation 
# Feed the source data, no need to split test/train (it folds it automatically)
# This internal method/function does the same thing as the FOR loop you wrote above
from sklearn.model_selection import cross_val_score
cross_val_score(LogisticRegression(), digits.data, digits.target)


array([0.89534884, 0.94991653, 0.90939597])

In [37]:
# Check for SVM
cross_val_score(SVC(), digits.data, digits.target)

array([0.39368771, 0.41068447, 0.45973154])

In [38]:
# Check for RF
cross_val_score(RandomForestClassifier(n_estimators=40), digits.data, digits.target)

array([0.93687708, 0.93989983, 0.93120805])